파이썬 기본 구현인 CPython은 메모리 관리를 위해 참조 카운팅을 사용한다.

어떤 객체를 가리키는 객체가 모두 없어지면 메모리에서 삭제되고 메모리 공간을 다른 데이터에 내어 줄 수 있다.

CPython에는 순환 탐지기가 들어 있으므로 자기 자신을 참조하는 (A -> B -> C -> A) 처럼 고리 모양으로 서로를 참조하는 객체의 메모리도

언젠간 쓰레기 수집(garbage collect) 된다

CPython 런타임이 알아서 메모리 관리를 해주고, 실전에서는 더 이상 사용하지 않는 쓸모없는 참조를 유지하기 때문에

언젠가는 결국 메모리를 소진하게 되는 경우가 있다. 

메모리 사용을 디버깅하는 첫 번째 방법은 gc 내장 모듈을 사용해서 현재 쓰레기 수집기가 알고 있는 모든 객체를 나열 시키는 거ㅅdlek.

In [1]:
import os

class MyObject:
    def __init__(self):
        self.data = os.urandom(100)

def get_data():
    values = []
    for _ in range(100):
        obj = MyObject()
        values.append(obj)
    return values

def run():
    deep_values = []
    for _ in range(100):
        deep_values.append(get_data())
    return deep_values

In [3]:
# gc 내장 모듈을 사용해 실행중 생성한 객체의 개수와 생성한 객체 중 일부를 출력하는 프로그램을 실행한다.

import gc
found_objects = gc.get_objects()
print('이전:', len(found_objects))

hold_reference = run()

found_objects = gc.get_objects()
print('이후:', len(found_objects))

for obj in found_objects[:3]:
    print(repr(obj)[:100])

이전: 82392
이후: 92387


gc.get_objects()의 문제점은 객체가 어떻게 할당됐는지를 알려주지 않는다는 것이다.

복잡한 프로그램에서는 특정 클래스에 속하는 객체가 여러 다른 방식으로 할당될 수 있다.

객체 전체 개수를 아는 것은 메모리를 누수시키는 객체를 할당한 코드를 알아내는 것 만큼 중요하지 않다.

tracemalloc은 자신이 할당된 장소와 연결해 메모리 사용의 이전과 이후의 스냅샷을 만들어 서로 비교하여 어떤 부분이 변경됬는지 알 수 있다.

In [4]:
import tracemalloc

tracemalloc.start(10) # 스택 깊이 설정 
time1 = tracemalloc.take_snapshot() # 이전 스냅샷

x = run() # 이 부분의 메모리 사용을 디버깅함
time2 = tracemalloc.take_snapshot() # 이후 스냅샷

stats = time2.compare_to(time1, 'lineno') # 두 스냅샷을 비교

for stat in stats[:3]:
    print(stat)

/tmp/ipykernel_649962/3143607866.py:5: size=2314 KiB (+2314 KiB), count=29985 (+29985), average=79 B
/tmp/ipykernel_649962/3143607866.py:10: size=469 KiB (+469 KiB), count=10000 (+10000), average=48 B
/tmp/ipykernel_649962/3143607866.py:11: size=84.4 KiB (+84.4 KiB), count=100 (+100), average=864 B


In [7]:
import tracemalloc
tracemalloc.start(10)

time1 = tracemalloc.take_snapshot()

x = run()
time2 = tracemalloc.take_snapshot()

stats = time2.compare_to(time1, 'traceback')
top = stats[0]

print('기장 많이 사용하는 부분은:')
print('\n'.join(top.traceback.format()))

기장 많이 사용하는 부분은:
  File "/root/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3046
    result = self._run_cell(
  File "/root/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3101
    result = runner(coro)
  File "/root/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/core/async_helpers.py", line 129
    coro.send(None)
  File "/root/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3306
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "/root/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3488
    if await self.run_code(code, result, async_=asy):
  File "/root/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3548
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_649962/1250986514.py", line 9
    stats = time2.compare_to(t